In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
ml=tf.constant([[3,3]])
m2=tf.constant([[2],[1]])
product = tf.matmul(ml,m2)
print(product)

Tensor("MatMul:0", shape=(1, 1), dtype=int32)


In [ ]:
sess = tf.Session()
#run触发图中3个op
result = sess.run(product)
print (result)
sess.close()

[[9]]


In [7]:
with tf.Session() as sess:
    result = sess.run(product)
    print (result)

[[9]]


In [8]:
#变量
x=tf.Variable([1,2])
a=tf.constant([3,3])
sub = tf.subtract(x,a)
add = tf.add(x,sub)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(sub))
    print(sess.run(add))

[-2 -1]
[-1  1]


In [11]:
#循环
#变量初始值=0
state=tf.Variable(0,  name='counter')
new_value= tf.add(state, 1)
update= tf.assign(state, new_value)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(state))
    for i in range(5):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4
5


In [14]:
#fetch and feed
#fetch 同时运行多个op
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(4.0)
add=tf.add(input1, input2)
mul=tf.multiply(input1, add)
with tf.Session() as sess:
    result = sess.run([mul,add])
    print(result)

[15.0, 5.0]


In [16]:
#feed
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)
with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7], input2:[8]}))

[ 56.]


In [18]:
#simple demo
x_data=np.random.rand(100)
y_data=x_data*0.1+0.2

b=tf.Variable(0.)
k=tf.Variable(0.)
y=k*x_data+b

loss=tf.reduce_mean(tf.square(y_data-y))
optimizer=tf.train.GradientDescentOptimizer(0.2)
train = optimizer.minimize(loss)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(200):
        sess.run(train)
        if(step % 20 ==0):
            print(step,sess.run([k,b]))

0 [0.061942585, 0.10297129]
20 [0.10995129, 0.19397114]
40 [0.1065317, 0.19604288]
60 [0.10428718, 0.19740267]
80 [0.10281397, 0.19829521]
100 [0.10184699, 0.19888103]
120 [0.1012123, 0.19926555]
140 [0.1007957, 0.19951794]
160 [0.10052228, 0.19968359]
180 [0.1003428, 0.19979233]


In [5]:
mnist = input_data.read_data_sets('../MNIST/MNIST_data', one_hot=True)

#定义批从
batch_size = 100
n_batchs = mnist.train.num_examples // batch_size

#参数
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('histogram', var)

#初始化权重
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

#初始化偏置
def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

#卷积层
def conv2d(x, W):
    #input tensor [batch, in_height, in_width, in_channels]
    #strides : 步长 strides[0]=strides[3]=1, strides[1]:x步长， strides[2]:y 步长
    #padding "SAME" "VALID"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1, x, y, 1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('input'):
    #nn
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    y = tf.placeholder(tf.float32, [None, 10], name='y_input')
    with tf.name_scope("x_image"):
    #CNN需要2维数据 [batch, height, width, channels]
        x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')

#L1
with tf.name_scope('conv1'):
    #初始化weight和bias
    with tf.name_scope('w_conv1'):
        w_conv1 = weight_variable([5, 5, 1, 32], name='w_conv1') # 5 * 5的窗口，从一个平面上抽32个平面
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32], name='b_conv1')  #每个平面加一个bias
    #卷积
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image, w_conv1) + b_conv1  #x_image和权值向量卷积，加上偏置，再加relu
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    #池化
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)   #池化

#L2
with tf.name_scope('conv2'):
    #初始化weight & bias
    with tf.name_scope('w_conv2'):
        w_conv2 = weight_variable([5, 5, 32, 64], name='w_conv2') # 5 * 5的窗口，从一个平面上抽32个平面
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64], name='b_conv2')  #每个平面加一个bias
    #卷积
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1, w_conv2) + b_conv2
    with tf.name_scope('relu'):
        h_conv2 = tf.nn.relu(conv2d_2)
    #池化
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

#1st:28*28 卷积后:28*28, 池化后：14*14
#2nd:14*14  卷积后:14*14, 池化后: 7*7
#最后得到 64张 7*7 的平面

#全连接层
with tf.name_scope('fc1'):
#L1
    #初始化weight & bias
    with tf.name_scope('w_fc1'):
        w_fc_1 = weight_variable([7*7*64, 1024], name='w_fc1')  
    with tf.name_scope('b_fc1'):
        b_fc_1 = weight_variable([1024], name='b_fc1')
    #把7*7*64的结果平铺为1层
    with tf.name_scope('h_pool_flat1'):  
        h_pool2_1d = tf.reshape(h_pool2, [-1, 7 * 7 * 64], name='h_pool_flat1')
    #tanh(wx + b)
    with tf.name_scope('fc_out'):
        h_fc_a_1 = tf.matmul(h_pool2_1d, w_fc_1) + b_fc_1
    with tf.name_scope('relu'):
        h_fc_out_1 = tf.nn.relu(h_fc_a_1)
    #加一层drop-out
    with tf.name_scope('drop_out'):
        drop_out = tf.placeholder(tf.float32, name='dropout')  #dropout
    with tf.name_scope('out_1'):
        L1_drop = tf.nn.dropout(h_fc_out_1, drop_out, name='out_1')

#L2
with tf.name_scope('fc2'):
    #初始化weight & bias
    with tf.name_scope('w_fc2'):
        w_fc_2 = weight_variable([1024, 10], name='w_fc2')
    with tf.name_scope('b_fc2'):
        b_fc_2 = bias_variable([10], name='b_fc2')
    # softmax(w*x + b)
    with tf.name_scope('fc_out2'):
        h_fc_out_2 = tf.matmul(L1_drop, w_fc_2) + b_fc_2
    with tf.name_scope('softmax'):
        predict = tf.nn.softmax(h_fc_out_2)

#loss function
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predict), name='cross_entropy')
    tf.summary.scalar('cross_entroy', cross_entropy)
with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_predict'):
        correct_predict = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        
merge = tf.summary.merge_all()
        
#run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train', sess.graph)
    test_writer = tf.summary.FileWriter('logs/test', sess.graph)
    for epoch in range(200):
        #train
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={x:batch_xs, y:batch_ys, drop_out:0.7})

        summary = sess.run(merge, feed_dict={x:batch_xs, y:batch_ys, drop_out:0.7})
        train_writer.add_summary(summary, epoch)
        #test
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)            
        summary = sess.run(merge, feed_dict={x:batch_xs, y:batch_ys, drop_out:0.7})
        test_writer.add_summary(summary, epoch)

        if(epoch % 100 ==0):
            test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y: mnist.test.labels, drop_out:1.0})
            train_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y: mnist.test.labels, drop_out:1.0})
            print("Iter: " + str(epoch)+" test acc: "+ str(test_acc) + " train acc: " + str(train_acc))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter: 0 test acc: 0.088 train acc: 0.088


KeyboardInterrupt: 